In [345]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import ui
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import date

In [321]:
url = 'https://manager.paypal.com/'
browser = webdriver.Firefox()
browser.get(url)

In [322]:
with open('credentials.txt') as file:
    partnerName = file.readline()
    vendorName = file.readline()
    userID = file.readline()
    password = file.readline()

In [323]:
elem1 = browser.find_element_by_name('partnerName')  # Find the search box
elem1.send_keys(partnerName)
elem2 = browser.find_element_by_name('vendorName')  # Find the search box
elem2.send_keys(vendorName)
elem3 = browser.find_element_by_name('userID')  # Find the search box
elem3.send_keys(userID)
elem4 = browser.find_element_by_name('password')  # Find the search box
elem4.send_keys(password + Keys.RETURN)

In [324]:
def page_is_loaded(driver):
    return driver.find_element_by_css_selector(".horizontalScroll") != None
wait = ui.WebDriverWait(browser, 10)
wait.until(page_is_loaded)
#browser.page_source

True

In [325]:
elem5 = browser.find_element_by_css_selector(".menu > dl:nth-child(1) > dd:nth-child(7) > a:nth-child(1) > span:nth-child(2)")
elem5.click()

In [326]:
elem6 = browser.find_element_by_link_text('Manage Profiles')
elem6.click()

In [327]:
elem7 = browser.find_element_by_css_selector("#manageProfile > form:nth-child(1) > table:nth-child(2) > tfoot:nth-child(5) > tr:nth-child(1) > td:nth-child(2) > input:nth-child(2)")
elem7.click()

In [342]:
data = []
headers = ['ProfileId', 'Profile Name', 'Last Name', 'First Name', 'Status', 'Account Number', \
           'Expiration Date', 'Amount', 'Frequency', 'Term', 'Established Date', 'Start Date', 'Next Payment Date', \
           'Deactivation Date', 'Profile Modified Date']
i = 0
while True:
    source = browser.page_source
    soup = BeautifulSoup(source, "lxml")
    table = soup.find("table")
    if len(table.findAll("tr")) < 2:
        break
    for row in table.findAll("tr"):
        cells = row.findAll("td")
        d = {}
        for header, cell  in zip(headers, cells):
            d[header] = cell.text
        data.append(d)
    browser.get('https://manager.paypal.com/searchProfiles.do?subaction=searchRbProfile&start=%d' % (i*50 + 51))
    wait = ui.WebDriverWait(browser, 10)
    wait.until(page_is_loaded)
    i = i + 1
    if i > 500:
        print('Didn\'t work')
        break

1


In [357]:
df = pd.DataFrame().from_dict(data)[headers]
df = df[df['ProfileId'].notnull()]
df.to_csv('%s_Payment.csv' % date.today().strftime('%y-%m-%d'))